In [170]:
import arcpy
from arcpy import env
from arcpy.sa import *
env.workspace = "C:\Rok3\teledetekcja\2\proj2"
image = "C:\Rok3\teledetekcja\2\pleiades_ms_20130620_Dworzec_Zach.img"
image2 = "C:\Rok3\teledetekcja\2\proj2\S2B_tile.TIF"

In [171]:
blue = arcpy.sa.ExtractBand("S2B_tile.TIF", [1])
green = arcpy.sa.ExtractBand("S2B_tile.TIF", [2])
red = arcpy.sa.ExtractBand("S2B_tile.TIF", [3])
nir = arcpy.sa.ExtractBand("S2B_tile.TIF", [4])
blue.save(r"c:\Rok3\teledetekcja\2\proj2\blue.tif")
green.save(r"c:\Rok3\teledetekcja\2\proj2\green.tif")
red.save(r"c:\Rok3\teledetekcja\2\proj2\red.tif")
nir.save(r"c:\Rok3\teledetekcja\2\proj2\nir.tif")

In [172]:
ndvi = arcpy.ia.RasterCalculator([nir,red], ['nir','red'],"(nir-red)/(nir+red)")
ndvi.save(r"c:\Rok3\teledetekcja\2\proj2\ndvi.tif")

In [173]:
blue_nir = arcpy.ia.RasterCalculator([blue, nir],['blue','nir'],"(blue+0) / (nir+0)")
blue_nir.save(r"c:\Rok3\teledetekcja\2\proj2\blue_nir.tif")

In [174]:
min1 = arcpy.management.GetRasterProperties(ndvi, "MINIMUM")[0]
max1 = arcpy.management.GetRasterProperties(ndvi, "MAXIMUM")[0]
print(f"min1: {min1}, max1: {max1}")

min1: -0,278904175718079, max1: 0,871574934477007


In [175]:
Reclass_ndvi = arcpy.sa.Reclassify(
    in_raster="ndvi.tif",
    reclass_field="VALUE",
    remap=f'{min1} 0,160000 0;0,160000 {max1} 1', 
#     remap="-0,757303 0,160000 0;0,160000 0,692967 1",
    missing_values="DATA"
)
Reclass_ndvi.save(r"C:\Rok3\teledetekcja\2\proj2\Reclass_ndvi.tif")

In [176]:
bluenir_minus_ndvirec = arcpy.ia.RasterCalculator(['blue_nir.tif','Reclass_ndvi.tif'],['bluenir','ReclassNDVI'],"bluenir-ReclassNDVI")
bluenir_minus_ndvirec.save(r"c:\Rok3\teledetekcja\2\proj2\bluenir_minus_ndvirec.tif")

In [177]:
min2 = arcpy.management.GetRasterProperties(bluenir_minus_ndvirec, "MINIMUM")[0]
max2 = arcpy.management.GetRasterProperties(bluenir_minus_ndvirec, "MAXIMUM")[0]
print(f"min2: {min2}, max2: {max2}")

min2: -0,923233609166136, max2: 1,72564734895191


In [179]:
bluenir_minus_ndvirec_reclass = arcpy.sa.Reclassify(
    in_raster="bluenir_minus_ndvirec.tif",
    reclass_field="Value",
    remap=f'{min2} 0,630000 0;0,630000 0,860000 1;0,860000 {max2} 0',
    missing_values="DATA"
)
bluenir_minus_ndvirec_reclass.save(r"C:\Rok3\teledetekcja\2\proj2\bluenir_minus_ndvirec_reclass.tif")

In [180]:
nir_blue = arcpy.ia.RasterCalculator([nir,blue], ['nir','blue'],"(nir+0)/(blue+0)")
nir_blue.save(r"c:\Rok3\teledetekcja\2\proj2\nir_blue.tif")

In [181]:
min3 = arcpy.management.GetRasterProperties(nir_blue, "MINIMUM")[0]
max3 = arcpy.management.GetRasterProperties(nir_blue, "MAXIMUM")[0]
print(f"min3: {min3}, max3: {max3}")

min3: 0,579492675955698, max3: 13,0265339966833


In [182]:
nir_blueReclass = arcpy.sa.Reclassify(
    in_raster="nir_blue.tif",
    reclass_field="Value",
    remap=f'{min3} 1,200000 0;1,200000 1,490000 1;1,490000 {max3} 0',
    missing_values="DATA"
)
nir_blueReclass.save(r"C:\Rok3\teledetekcja\2\proj2\nir_blueReclass.tif")

In [183]:
ironoxide = arcpy.ia.RasterCalculator([red,blue], ['red','blue'],"(red+0)/(blue+0)")
ironoxide.save(r"c:\Rok3\teledetekcja\2\proj2\ironoxide.tif")

In [184]:
min4 = arcpy.management.GetRasterProperties(ironoxide, "MINIMUM")[0]
max4 = arcpy.management.GetRasterProperties(ironoxide, "MAXIMUM")[0]
print(f"min4: {min4}, max4: {max4}")

min4: 0,533256217466744, max4: 5,65675467828762


In [185]:
ironReclass = arcpy.sa.Reclassify(
    in_raster="ironoxide.tif",
    reclass_field="Value",
    remap=f'{min4} 1,180000 1;1,180000 1,350000 0;1,350000 {max4} 1',
    missing_values="DATA"
)
ironReclass.save(r"C:\Rok3\teledetekcja\2\proj2\ironReclass.tif")

In [186]:
arcpy.conversion.RasterToPolygon(
    in_raster="bluenir_minus_ndvirec_reclass",
    out_polygon_features=r"C:\Rok3\teledetekcja\2\projekt.gdb\RasterT_bluenirndvi",
    simplify="SIMPLIFY",
    raster_field="Value",
    create_multipart_features="SINGLE_OUTER_PART",
    max_vertices_per_feature=None
)

<Result 'C:\\Rok3\\teledetekcja\\2\\projekt.gdb\\RasterT_bluenirndvi'>

In [187]:
bluenirndvi = arcpy.management.SelectLayerByAttribute(
    in_layer_or_view="RasterT_bluenirndvi",
    selection_type="NEW_SELECTION",
    where_clause="gridcode = 1",
    invert_where_clause=None
)
arcpy.management.CopyFeatures(bluenirndvi,
     out_feature_class=r"C:\Rok3\teledetekcja\2\proj2\bluenirndvi")

<Result 'C:\\Rok3\\teledetekcja\\2\\proj2\\bluenirndvi.shp'>

In [188]:
arcpy.conversion.RasterToPolygon(
    in_raster="nir_blueReclass",
    out_polygon_features=r"C:\Rok3\teledetekcja\2\proj2\RasterT_nir_blueReclass",
    simplify="SIMPLIFY",
    raster_field="Value",
    create_multipart_features="SINGLE_OUTER_PART",
    max_vertices_per_feature=None
)

<Result 'C:\\Rok3\\teledetekcja\\2\\proj2\\RasterT_nir_blueReclass.shp'>

In [189]:
RasterT_nirblue = arcpy.management.SelectLayerByAttribute(
    in_layer_or_view="RasterT_nir_blueReclass",
    selection_type="NEW_SELECTION",
    where_clause="gridcode = 1",
    invert_where_clause=None
)
arcpy.management.CopyFeatures(bluenirndvi,
     out_feature_class=r"C:\Rok3\teledetekcja\2\proj2\RasterT_nirblue")

<Result 'C:\\Rok3\\teledetekcja\\2\\proj2\\RasterT_nirblue.shp'>

In [190]:
arcpy.conversion.RasterToPolygon(
    in_raster="ironReclass",
    out_polygon_features=r"C:\Rok3\teledetekcja\2\projekt.gdb\RasterT_ironReclass",
    simplify="SIMPLIFY",
    raster_field="Value",
    create_multipart_features="SINGLE_OUTER_PART",
    max_vertices_per_feature=None
)

<Result 'C:\\Rok3\\teledetekcja\\2\\projekt.gdb\\RasterT_ironReclass'>

In [191]:
RasterT_ironoxide = arcpy.management.SelectLayerByAttribute(
    in_layer_or_view="RasterT_ironReclass",
    selection_type="NEW_SELECTION",
    where_clause="gridcode = 0",
    invert_where_clause=None
)
arcpy.management.CopyFeatures(bluenirndvi,
     out_feature_class=r"C:\Rok3\teledetekcja\2\proj2\RasterT_ironoxide")

<Result 'C:\\Rok3\\teledetekcja\\2\\proj2\\RasterT_ironoxide.shp'>

In [192]:
raster1i2 = arcpy.management.SelectLayerByLocation(
    in_layer="RasterT_nirblue",
    overlap_type="CONTAINS",
    select_features="bluenirndvi",
    search_distance=None,
    selection_type="NEW_SELECTION",
    invert_spatial_relationship="NOT_INVERT"
)
arcpy.management.CopyFeatures(raster1i2,
     out_feature_class=r"C:\Rok3\teledetekcja\2\proj2\raster1i2")

<Result 'C:\\Rok3\\teledetekcja\\2\\proj2\\raster1i2.shp'>

In [193]:
# rastryErase = arcpy.analysis.Erase(
#     in_features="raster1i2",
#     erase_features="RasterT_ironoxide",
#     out_feature_class=r"C:\Rok3\teledetekcja\2\proj2\rastryErase",
#     cluster_tolerance=None
# )
rastry = arcpy.management.SelectLayerByLocation(
    in_layer="raster1i2",
    overlap_type="CONTAINS",
    select_features="RasterT_ironoxide",
    search_distance=None,
    selection_type="NEW_SELECTION",
    invert_spatial_relationship="NOT_INVERT"
)
arcpy.management.CopyFeatures(rastry,
     out_feature_class=r"C:\Rok3\teledetekcja\2\proj2\rastry")

<Result 'C:\\Rok3\\teledetekcja\\2\\proj2\\rastry.shp'>

In [194]:
rastryMultipartToSingle = arcpy.management.MultipartToSinglepart(
    in_features="rastry",
    out_feature_class=r"C:\Rok3\teledetekcja\2\proj2\rastryEraseMultipartToSingle"
)

In [195]:
arcpy.management.CalculateGeometryAttributes(
    in_features="rastryEraseMultipartToSingle",
    geometry_property="area AREA;numholes HOLE_COUNT",
    length_unit="KILOMETERS",
    area_unit="SQUARE_METERS",
    coordinate_system=None,
    coordinate_format="SAME_AS_INPUT"
)

<Result 'rastryEraseMultipartToSingle'>

In [196]:
wieksze = arcpy.management.SelectLayerByAttribute(
    in_layer_or_view="rastryEraseMultipartToSingle",
    selection_type="NEW_SELECTION",
    where_clause="area > 20000",
    invert_where_clause=None
)
arcpy.management.CopyFeatures(wieksze,r"C:\Rok3\teledetekcja\2\proj2\rastryEraseMultipartToSingle_CopyFeatures")

<Result 'C:\\Rok3\\teledetekcja\\2\\proj2\\rastryEraseMultipartToSingle_CopyFeatures.shp'>

In [197]:
arcpy.management.PolygonToLine(
    in_features="rastryEraseMultipartToSingle_CopyFeatures",
    out_feature_class=r"C:\Rok3\teledetekcja\2\projekt.gdb\rastry_PolygonToLine1",
    neighbor_option="IDENTIFY_NEIGHBORS"
)

<Result 'C:\\Rok3\\teledetekcja\\2\\projekt.gdb\\rastry_PolygonToLine1'>

In [198]:
wynik = arcpy.management.SelectLayerByAttribute(
    in_layer_or_view="rastry_PolygonToLine1",
    selection_type="NEW_SELECTION",
    where_clause="Shape_Length > 1000",
    invert_where_clause=None
)
arcpy.management.CopyFeatures(wynik,r"C:\Rok3\teledetekcja\2\proj2\wynik")

<Result 'C:\\Rok3\\teledetekcja\\2\\proj2\\wynik.shp'>

In [199]:
podzielone = arcpy.management.SplitLine(
    in_features="wynik",
    out_feature_class=r"C:\Rok3\teledetekcja\2\projekt.gdb\wynik_SplitLine"
)
wynik2 = arcpy.management.SelectLayerByAttribute(
    in_layer_or_view=podzielone,
    selection_type="NEW_SELECTION",
    where_clause="Shape_Length > 35",
    invert_where_clause=None
)
arcpy.management.CopyFeatures(wynik2,r"C:\Rok3\teledetekcja\2\proj2\wynik2")


<Result 'C:\\Rok3\\teledetekcja\\2\\proj2\\wynik2.shp'>

In [200]:
zlaczone = arcpy.management.UnsplitLine(
    in_features="wynik2",
    out_feature_class=r"C:\Rok3\teledetekcja\2\projekt.gdb\wynik2_UnsplitLine",
    dissolve_field=None,
    statistics_fields=None,
    concatenation_separator=""
)
wynik3 = arcpy.management.SelectLayerByAttribute(
    in_layer_or_view=zlaczone,
    selection_type="NEW_SELECTION",
    where_clause="Shape_Length > 75",
    invert_where_clause=None
)
arcpy.management.CopyFeatures(wynik3,r"C:\Rok3\teledetekcja\2\proj2\wynik3")

<Result 'C:\\Rok3\\teledetekcja\\2\\proj2\\wynik3.shp'>